In [27]:
from typing import Dict, List
import sys
import importlib

import pandas as pd
import numpy as np

REPO_RELATIVE_PATH = '..'
METRICS_MODULE_PATH = 'src.metrics.metrics'

if REPO_RELATIVE_PATH not in sys.path:
    sys.path.append(REPO_RELATIVE_PATH)

In [2]:
items = pd.read_csv(
    f'{REPO_RELATIVE_PATH}/datasets/ml-1m/ml-1m.item',
    sep='\t',
    header=0,
    names=['item_id', 'movie_title', 'release_year', 'genre']
)
datasets = {
    'random': pd.read_csv('splits/random/val.csv', sep='\t', header=0),
    'by_user': pd.read_csv('splits/by_user/val.csv', sep='\t', header=0),
    'lol': pd.read_csv('splits/leave_one_last/val.csv', sep='\t', header=0),
    't_user': pd.read_csv('splits/temporal_user/val.csv', sep='\t', header=0),
    't_global': pd.read_csv('splits/temporal_global/val.csv', sep='\t', header=0),
}

Составим выборку случайных фильмов для тестирования метрик на разных сплитах

In [3]:
random_recommendations = items.sample(frac=1, random_state=42).reset_index()['item_id']
random_recommendations.head()

0    1365
1    2706
2    3667
3    3684
4    1881
Name: item_id, dtype: int64

In [4]:
def prepare_labels(
    df: pd.DataFrame,
    threshhold: int | float,
    user_id_col: str = 'user_id',
    item_id_col: str = 'item_id',
    rating_col: str = 'rating'
) -> pd.DataFrame:
    return (
        df
        .sort_values([user_id_col, rating_col], ascending=[True, False])
        .groupby(user_id_col)
        .agg(
            items=(item_id_col, lambda x: list(x)),
            ratings=(rating_col, lambda x: list(x))
        )
        .apply(lambda x: [i for (i, r) in zip(x['items'], x['ratings']) if r > threshhold], axis=1)
    )

In [11]:
random_labels = prepare_labels(datasets['random'], 3)
random_labels.head()

user_id
1                       [1035, 3105, 1193, 1836, 2018]
2           [1945, 2002, 1357, 1957, 3468, 3451, 3068]
3                             [1259, 1196, 1049, 1394]
4                                                   []
5    [2427, 3083, 2997, 1175, 2289, 348, 1392, 506,...
dtype: object

Посчитаем метрики для разных сплитов на рандомной подборке

In [35]:
def compute_metrics(
    datasets: Dict[str, pd.DataFrame],
    metric_list_with_kwargs: Dict[str, Dict],
    prepare_labels_kwargs: Dict = dict()
) -> None:
    metrics_module = importlib.import_module(METRICS_MODULE_PATH)
    metrics_module = importlib.reload(metrics_module)
    for df_name, df in datasets.items():
        df_labels = prepare_labels(df, **prepare_labels_kwargs)
        for metric_name, kwargs in metric_list_with_kwargs.items():
            metric = getattr(metrics_module, metric_name)
            result = df_labels.apply(lambda x: metric(random_recommendations, x, **kwargs)).mean()
            print(f'{df_name} - {metric_name}({kwargs}): {result}')
        print('======================')

In [34]:
f'{ {"k": 10}.items() }'

"dict_items([('k', 10)])"

In [36]:
compute_metrics(
    datasets,
    {
        'precision_at_k': {'k': 10},
        'recall_at_k': {'k': 10},
        'f_beta_score_at_k': {'k': 10, 'beta': 1},
        'f1_score_at_k': {'k': 10},
    },
    {'threshhold': 3}
)

random - precision_at_k({'k': 10}): 0.002046636470390874
random - recall_at_k({'k': 10}): 0.002795959045627951
random - f_beta_score_at_k({'k': 10, 'beta': 1}): 0.001882293666513664
random - f1_score_at_k({'k': 10}): 0.001882293666513664
by_user - precision_at_k({'k': 10}): 0.019867549668874177
by_user - recall_at_k({'k': 10}): 0.003730259055027287
by_user - f_beta_score_at_k({'k': 10, 'beta': 1}): 0.004720854254498409
by_user - f1_score_at_k({'k': 10}): 0.004720854254498409
lol - precision_at_k({'k': 10}): 0.00029308323563892143
lol - recall_at_k({'k': 10}): 0.0029308323563892145
lol - f_beta_score_at_k({'k': 10, 'beta': 1}): 0.0005328786102525844
lol - f1_score_at_k({'k': 10}): 0.0005328786102525844
t_user - precision_at_k({'k': 10}): 0.001583476764199656
t_user - recall_at_k({'k': 10}): 0.0019312661802910493
t_user - f_beta_score_at_k({'k': 10, 'beta': 1}): 0.0013784796162528202
t_user - f1_score_at_k({'k': 10}): 0.0013784796162528202
t_global - precision_at_k({'k': 10}): 0.01049382

Оказывается случайная подборка лучше всего сработала для холодного старта, когда мы еще ничего не знаем о пользователе. С другой стороны в этом датасете просто самые длинные списки фильмов, так как брали всю историю просмотров.